Get Started (detailed)

In [1]:
import torch
import librosa
import numpy as np
from modules import feature_extractor, DetectionNet, BreathDetector

In [4]:
!pip install config torch==2.2.2 torchaudio==2.2.2 librosa==0.10.0 numpy==1.23.0 intervaltree==3.1.0


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# model loading
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = DetectionNet().to(device)
# Load checkpoint with correct map_location
checkpoint = torch.load("respiro-en.pt", map_location=device)
model.load_state_dict(checkpoint["model"])
model.eval()

cpu


DetectionNet(
  (downsampling): Conv2dDownsampling(
    (conv1): Sequential(
      (0): Conv2d(3, 1, kernel_size=(3, 3), stride=(2, 2))
      (1): ReLU()
    )
    (conv2): Sequential(
      (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
      (1): ReLU()
    )
  )
  (upsampling): Conv1dUpsampling(
    (deconv): Sequential(
      (0): ConvTranspose1d(128, 128, kernel_size=(3,), stride=(2,))
      (1): ReLU()
      (2): ConvTranspose1d(128, 128, kernel_size=(3,), stride=(2,))
      (3): ReLU()
    )
  )
  (linear): Linear(in_features=31, out_features=128, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (conformer): Conformer(
    (conformer_layers): ModuleList(
      (0-7): 8 x ConformerLayer(
        (ffn1): _FeedForwardModule(
          (sequential): Sequential(
            (0): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (1): Linear(in_features=128, out_features=256, bias=True)
            (2): SiLU()
            (3): Dropout(p=0.1, inplace=False)


In [3]:
wav_path = "/Users/user/Desktop/breath_speech/Respiro-en/samples/Shyam Nagar 24.wav"
#wav_path = "samples/train-clean-360_14_208_000001_000000.wav"
#wav_path = "samples/train-other-500_20_205_000002_000002.wav"
wav, sr = librosa.load(wav_path, sr=16000)
feature, length = feature_extractor(wav)
feature, length = feature.to(device), length.to(device)
output = model(feature, length)

# 0.064 is the threshold obtained from our validation set
# You can try more strict thresholds like 0.5 or 0.9
threshold = 0.064

# min_length: length threshold to avoid too short detected breath, which tends to be the end part of speech
# default: 20 ms
min_length = 20

prediction = (output[0] > 0.064).nonzero().squeeze().tolist()
if isinstance(prediction, list) and len(prediction)>1:
    diffs = np.diff(prediction)
    splits = np.where(diffs != 1)[0] + 1
    splits = np.split(prediction, splits)
    splits = list(filter(lambda split: len(split)>min_length, splits))
    if len(splits)>1:
        for split in splits:
            print(split)
# The segments of breath are printed
# 229 means 229 ms

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55]
[195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212
 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230
 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248
 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266
 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284
 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302
 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320
 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338
 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356
 357 358 359 360 361 362]
[592 593 594 595 596 597 598 599 600 601 602 603 604 605 606 607 608 609
 610 611 612 613 614 615 616 617 618 619 620 621 622 623 624 625 626]
[7

In [4]:
start = splits[6][0]
end = splits[6][-1]

from IPython.display import Audio
Audio(data=wav[int((start*0.01)*sr):int((end*0.01)*sr)], rate=sr) 

Get Started (quick)

In [5]:
import torch
from modules import DetectionNet, BreathDetector

# model loading
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DetectionNet().to(device)
checkpoint = torch.load("respiro-en.pt", map_location=device)
model.load_state_dict(checkpoint["model"])
model.eval()

detector = BreathDetector(model) # Args: model, device=None

tree = detector("/Users/user/Desktop/breath_speech/Respiro-en/samples/Shyam Nagar 24.wav") # Args: wav_path, threshold=0.064, min_length=20
print(tree)

IntervalTree([Interval(0.0, 0.55), Interval(1.95, 3.62), Interval(5.92, 6.26), Interval(7.28, 8.69), Interval(11.07, 14.12), Interval(14.39, 14.75), Interval(15.26, 16.3)])


In [6]:
print(tree[2.6:5.2])

{Interval(1.95, 3.62)}


In [7]:
print(sorted(tree))

[Interval(0.0, 0.55), Interval(1.95, 3.62), Interval(5.92, 6.26), Interval(7.28, 8.69), Interval(11.07, 14.12), Interval(14.39, 14.75), Interval(15.26, 16.3)]
